## This scrips evalute Terminos Lagoon Delft3D model performance, using salinity and temperature data field

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import gsw as gsw
import matplotlib.dates as mdates

In [ ]:
# read in the csv file
def read_csv_file(filename):
    # read in the csv file
    df = pd.read_csv(filename, sep=",", header=0, decimal=".", encoding="utf-8") 
    return df

In [ ]:
# set folder and file names
sal_temp_terminos_file = "..\data\salinity_temperature_climatological_year_terminos_lagoon.csv"


In [ ]:
# Read salinity and temperature data from csv file
sal_temp_terminos_rawdata = read_csv_file(sal_temp_terminos_file)

In [ ]:
# print the column names
for col in sal_temp_terminos_rawdata.columns:
    print(col)

In [ ]:
# select the columns of interest
terminos_sal_temp = sal_temp_terminos_rawdata.copy()[
    [
        "Time_model",
        "salinity_ppt_marina_layer1",
        "temperature_C_marina_layer1",
        "Conductivity_microsiemens_cm_average",
        "Temp_C_average",
    ]
]

In [ ]:
# set time as index
terminos_sal_temp["Time_model"] = pd.to_datetime(
    terminos_sal_temp["Time_model"], format="%d/%m/%Y %H:%M"
    )

terminos_sal_temp = terminos_sal_temp.set_index(
    terminos_sal_temp["Time_model"]
    )

In [ ]:
months = [
    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December",
]

In [ ]:
# Calculate salinity from hobo conductivity

terminos_sal_temp["sea_level_preasure"] = 0

terminos_sal_temp["salinity_psu"] = gsw.conversions.SP_from_C(
    terminos_sal_temp["Conductivity_microsiemens_cm_average"] / 1000,
    terminos_sal_temp["Temp_C_average"],
    terminos_sal_temp["sea_level_preasure"],
)

## Model performance functions

In [ ]:
# Calculate basic statistics
def bias(predictions, targets):
    difference = predictions - targets
    bias_val = difference.mean()
    return bias_val

In [ ]:
# Mean Square Error (MSE)
def mse(predictions, targets):  
    differences_squared = (predictions - targets) ** 2
    mse_val = differences_squared.mean()
    return mse_val

In [ ]:
# Root Mean Square Error (RMSE)
def rmse(predictions, targets):  
    differences_squared = (predictions - targets) ** 2
    rmse_val = np.sqrt(differences_squared.mean())
    return rmse_val

In [ ]:
# Mean Absolute Error (MAE)
def mae(predictions, targets):  
    absoluteDifference = np.absolute(predictions - targets)
    mae_val = absoluteDifference.mean()
    return mae_val

In [ ]:
# Absolute Maximun Error (AME)
def ame(predictions, targets):  
    difference = predictions - targets
    ame_val = max(np.abs(difference))
    return ame_val

In [ ]:
# Model skill score (skill)
def model_skill(predictions, targets):
    differences = predictions - targets
    differencesSquared = abs(differences) ** 2
    sumDifferencesSquared = sum(differencesSquared)
    meanObservations = np.mean(targets)
    modelMinusMeanObservations = np.absolute(list(np.asarray(predictions) - meanObservations))
    observationsMinusMeanObservations = np.absolute(
        list(np.asarray(targets) - meanObservations)
    )
    denominator = sum(
        np.square(modelMinusMeanObservations + observationsMinusMeanObservations)
    )
    skill = 1 - sumDifferencesSquared / denominator
    return skill

In [ ]:
# Willmott agreement index (ia)
def willmott_agreement(predictions, targets):
    """
        index of agreement Willmott (1981, 1982) 
        input:
        s: simulated
        o: observed
    output:
        ia: index of agreement
    """
    ia = 1 - (np.sum((targets - predictions) ** 2)) / (
        np.sum(
            (
                np.abs(predictions - np.mean(targets))
                + np.abs(targets - np.mean(targets))
            )
            ** 2
        )
    )
    return ia


In [ ]:
def run_performance(predictions, targets):
    from tabulate import tabulate

    performance_metrics = {
        "Bias": bias,
        "Mean Square Error": mse,
        "Root Mean Square Error": rmse,
        "Mean Absolute Error": mae,
        "Absolute Maximum Error": ame,
        "Model Skill": model_skill,
        "Willmott Agreement": willmott_agreement
    }

    performance_results = [[name, round(func(predictions, targets), 1)] for name, func in performance_metrics.items()]
    
    table = tabulate(performance_results, headers=["Analysis", "Results"], tablefmt="simple_outline")
    print(table)

In [ ]:
# Salinity model performance analysis
print("Salinity model performance analysis \n ")
run_performance(terminos_sal_temp["salinity_ppt_marina_layer1"], terminos_sal_temp["salinity_psu"])

In [ ]:
print("Temperature model performance analysis\n")
run_performance(terminos_sal_temp["temperature_C_marina_layer1"], terminos_sal_temp["Temp_C_average"])